In [37]:
%%configure -f
{"driverMemory": "4G", "driverCores": 2, "executorMemory": "12G", "executorCores": 6, "numExecutors": 3}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,None,pyspark,idle,,,None,✔


In [53]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy', 'seaborn', 'statsmodels', 'pyarrow'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
from typing import Optional
import json
from pyspark.sql import SparkSession, DataFrame

spark: SparkSession

def load_distance_calculation_df(aggregation_pipeline: Optional[List] = None) -> DataFrame:
    if aggregation_pipeline is None:
        aggregation_pipeline = []
    
    return (
        spark
        .read
        .format("mongodb")
        .option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner")
        .option("partitioner.options.partition.field", "_id")
        .option("partitioner.options.partition.size", "64")
        .option("partitioner.options.samples.per.partition", "2")
        .option("database", "enhancer3d")
        .option("collection", "distance_calculation")
        .option("aggregation.pipeline", json.dumps(aggregation_pipeline))
        .load()
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
links_gm12878_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/GM12878_EP_hg38_liftovered.parquet")
    .alias("links")
)

links_hffc6_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/HFFC6_EP_hg38_liftovered.parquet")
    .alias("links")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
%%pretty
links_gm12878_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

enh_id,gene_id,pval,qval
chr1:843600-844990,ENSG00000197049,1.180354,1.0
chr1:966620-966700,ENSG00000197049,0.642743,2.0
chr1:843600-844990,ENSG00000188976,1.016813,3.0
chr1:966620-966700,ENSG00000188976,2.58213,4.0
chr1:1013580-1013980,ENSG00000188976,2.073247,5.0


In [42]:
from pyspark.sql import functions as F, types as T
import numpy as np

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
gm12878_neanderthal_df_ref = (
    load_distance_calculation_df([
        {
            '$match': {
                '_id.project_id': '8k_models_project_GM12878',
                '_id.ensemble_id': {'$regex': '^models3D_GM12878_Nean_models3D_GM12878_Nean_results'},
            }
        }
    ])
    .where(
        F.col('gene_type') == 'protein_coding'
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
    # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("gm12878")
    .cache()
)

hffc6_neanderthal_df_ref = (
    load_distance_calculation_df([
        {
            '$match': {
                '_id.project_id': '8k_models_project_HFFC6',
                '_id.ensemble_id': {'$regex': '^models3D_HFFC6_Nean_models3D_HFFC6_Nean_results'},
            }
        }
    ])
    .where(
        F.col('gene_type') == 'protein_coding'
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
     # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("hffc6")
    .cache()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<stdin>:20: SyntaxWarning: invalid escape sequence '\.'
<stdin>:44: SyntaxWarning: invalid escape sequence '\.'

In [48]:
%%pretty
gm12878_neanderthal_df_ref.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

region_id,gene_id,enh_id,avg_dist
chr10:10300866-12...,,chr10:10854747-10...,10.029472351074219
chr10:10300866-12...,,chr10:10869487-10...,12.780106544494629
chr10:10300866-12...,,chr10:11074217-11...,22.668201446533203
chr10:10300866-12...,,chr10:11080347-11...,22.45758628845215
chr10:10300866-12...,,chr10:11091167-11...,22.26520347595215


In [50]:
gm12878_existing_links_df = (
    gm12878_neanderthal_df_ref
    .join(
        other=links_gm12878_df,
        on=F.expr("gm12878.gene_id = links.gene_id AND gm12878.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        gm12878_neanderthal_df_ref.region_id,
        gm12878_neanderthal_df_ref.gene_id,
        gm12878_neanderthal_df_ref.enh_id,
        gm12878_neanderthal_df_ref.avg_dist
    )
)

hffc6_existing_links_df = (
    hffc6_neanderthal_df_ref
    .join(
        other=links_hffc6_df,
        on=F.expr("hffc6.gene_id = links.gene_id AND hffc6.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        hffc6_neanderthal_df_ref.region_id,
        hffc6_neanderthal_df_ref.gene_id,
        hffc6_neanderthal_df_ref.enh_id,
        hffc6_neanderthal_df_ref.avg_dist
    )
)

gm12878_hffc6_common_links_df = (
    gm12878_existing_links_df
    .join(
        other=hffc6_existing_links_df,
        on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
        how="inner"
    )
    .select(
        gm12878_existing_links_df.region_id,
        gm12878_existing_links_df.gene_id,
        gm12878_existing_links_df.enh_id,
        gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist'),
        hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
    )
)

gm12878_hffc6_gm12878_only_links_df = (
    gm12878_existing_links_df
    .join(
        other=hffc6_existing_links_df,
        on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
        how="left_anti"
    )
    .select(
        gm12878_existing_links_df.region_id,
        gm12878_existing_links_df.gene_id,
        gm12878_existing_links_df.enh_id,
        gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist')
    )
)

gm12878_hffc6_hffc6_only_links_df = (
    hffc6_existing_links_df
    .join(
        other=gm12878_existing_links_df,
        on=F.expr("hffc6.gene_id = gm12878.gene_id AND hffc6.enh_id = gm12878.enh_id"),
        how="left_anti"
    )
    .select(
        hffc6_existing_links_df.region_id,
        hffc6_existing_links_df.gene_id,
        hffc6_existing_links_df.enh_id,
        hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
gm12878_existing_links_df.count(), hffc6_existing_links_df.count(), gm12878_hffc6_common_links_df.count(), gm12878_hffc6_gm12878_only_links_df.count(), gm12878_hffc6_hffc6_only_links_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(13840, 20416, 3656, 10184, 16760)

In [54]:
import os

# write all to csv into /work/playground/links/experiment_1
os.makedirs("/work/playground/links/experiment_1", exist_ok=True)

gm12878_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_1/gm12878_neanderthal_df_ref.parquet", index=False)
hffc6_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_1/hffc6_neanderthal_df_ref.parquet", index=False)

gm12878_hffc6_common_links_df.toPandas().to_csv("/work/playground/links/experiment_1/gm12878_hffc6_common_links.csv", index=False)
gm12878_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_1/gm12878_existing_links.csv", index=False)
hffc6_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_1/hffc6_existing_links.csv", index=False)
gm12878_hffc6_gm12878_only_links_df.toPandas().to_csv("/work/playground/links/experiment_1/gm12878_hffc6_gm12878_only_links.csv", index=False)
gm12878_hffc6_hffc6_only_links_df.toPandas().to_csv("/work/playground/links/experiment_1/gm12878_hffc6_hffc6_only_links.csv", index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…